In [ ]:
!pip install Flask Flask-Cors

!pip install -q git+https://github.com/huggingface/transformers
!pip install -q git+https://github.com/huggingface/accelerate

!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # spécifie l'index pour une installation potentiellement plus stable avec CUDA
!pip install -q huggingface_hub sentencepiece protobuf

!pip install -q moviepy opencv-python Pillow librosa soundfile decord

!pip install qwen-vl-utils[decord]==0.0.8 einops timm bitsandbytes

!pip install langdetect

!pip install python-magic-bin # 'python-magic' nécessite souvent l'installation de 'python-magic-bin' sur Colab
!pip install numpy

!apt-get update && apt-get install -qq -y ffmpeg libmagic1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
 # Imports et Configuration Initiale
import os
import json
import logging
import time
import uuid # Peut être utile pour des noms uniques si besoin
from moviepy.editor import VideoFileClip
from PIL import Image
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor, AutoModelForCausalLM # Qwen 2.5 VL 3B
from transformers import pipeline # Whisper
from qwen_vl_utils import process_vision_info
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
import cv2
from google.colab import files # Pour l'upload
import warnings

In [ ]:
# Dossier temporaire dans Colab
TEMP_FOLDER = "/content/drive/MyDrive/Personas_project/Data/temp_files"
os.makedirs(TEMP_FOLDER, exist_ok=True)

Prompt.py

In [ ]:
prompt = """
Act as an expert AI specialized in analyzing multimedia content to extract marketing insights for user persona creation.
Your analysis must be objective and based solely on the provided information.
You are analyzing content from a user-generated video uploaded to a congratulatory video creation platform.

Analyze the provided image and the transcribed text.
Your task is to extract **a single most likely value** for each field below, to help marketing teams build accurate personas.

Output your results strictly in the following JSON structure (field names must match exactly):

Rules:
- For every field below, you must select **only one value** from the predefined list.
  If you are **uncertain** or the content does not clearly support a choice, return `"other"` — do not guess or list multiple options.
- Do **not list more than one category** for any field — only the most relevant one, or `"other"` if unclear.

- For `number_of_people`, base your answer solely on the visual image. Count how many individual people are clearly visible.
- If there is **more than one person**, return attributes (`gender`, `age`, `attire`) for each person separately, in a list.

- For `age`, estimate in years and classify into one of the following ranges (return the range label as-is):

    - "0–2 years"
    - "3–12 years"
    - "13–17 years"
    - "18–24 years"
    - "25–34 years"
    - "35–49 years"
    - "50–64 years"
    - ">=65 years"

- Gender and attire must also be based on visible cues only. If uncertain, return `"other"`.

JSON format:

QA = {
  "number_of_people": "1 | 2 | 3 | 4 | 5 | >5",
  "people": [
    {
      "gender": "female | male | other",
      "age": "0–2 years | 3–12 years | 13–17 years | 18–24 years | 25–34 years | 35–49 years | 50–64 years | >=65 years | other",
      "attire": "casual | formal | business | sport | party | uniform | other"
    }
  ],
  "recording_location": "home | living room | kitchen | bedroom | garden | outdoor | party | office | meeting room | open space | stage | car | other",
  "motivation": "personal | professional | other",
  "occasion": "birthday | wedding | engagement | birth | baptism | love message | mother's day | father's day | farewell | get well | fun video | graduation | work anniversary | promotion | new job | success | team celebration | product presentation | company presentation | conference | retirement | new year | christmas | valentine's day | halloween | easter | international women's day | other",
  "viral_mood": "happy | excited | fun | proud | grateful | emotional | festive | nostalgic | loving | motivated | inspirational | formal | other",
  "relationship": "mother | father | sister | brother | spouse | partner | husband | wife | child | relative | boyfriend | girlfriend | fiancé | friend | best friend | colleague | boss | employee | teacher | student | client | lead | other"
}
"""

Utils.py

In [ ]:
# --- Initialisation Speech-to-Text (Whisper)---

WHISPER_MODEL = "openai/whisper-base"
print(f"Chargement du modèle de transcription: {WHISPER_MODEL}...")
try:
    transcription_pipeline = pipeline(
        "automatic-speech-recognition",
        model=WHISPER_MODEL,
        device=device
    )
    print("Modèle de transcription chargé.")
except Exception as e:
    print(f"Erreur lors du chargement du modèle Whisper: {e}")
    transcription_pipeline = None

Chargement du modèle de transcription: openai/whisper-base...


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

Device set to use cuda


Modèle de transcription chargé.


In [ ]:
# --- Initialisation VLM (QWEN2.5 VL 3B)---
QWEN_MODEL = "Qwen/Qwen2.5-VL-3B-Instruct"
qwen_vl_processor = AutoProcessor.from_pretrained(QWEN_MODEL, trust_remote_code=True)
qwen_vl_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    QWEN_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# --- Fonctions Utilitaires Video Preprosessing ---

def generate_unique_filename(directory, extension):
    """Génère un nom de fichier unique dans un dossier."""
    while True:
        filename = str(uuid.uuid4()) + extension
        filepath = os.path.join(directory, filename)
        if not os.path.exists(filepath):
            return filepath

def extract_middle_frame(video_path, output_dir=TEMP_FOLDER):
    """Extrait l'image du milieu et la retourne en tant qu'objet PIL Image."""
    print(f"Extraction de l'image du milieu de : {video_path}")
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print("Erreur: Impossible d'ouvrir la vidéo.")
            return None
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total_frames == 0:
             print("Erreur: La vidéo ne contient aucune image.")
             cap.release()
             return None
        middle_frame_index = total_frames // 2
        cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame_index)
        ret, frame = cap.read()
        cap.release()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(frame_rgb)
            print("Image du milieu extraite avec succès.")
            return image
        else:
            print("Erreur: Impossible de lire l'image du milieu.")
            return None
    except Exception as e:
        print(f"Erreur lors de l'extraction de l'image : {e}")
        return None

def extract_audio(video_path, output_dir=TEMP_FOLDER):
    """Extrait l'audio et retourne le chemin du fichier audio temporaire."""
    print(f"Extraction de l'audio de : {video_path}")
    audio_path = generate_unique_filename(output_dir, ".wav")
    try:
        with VideoFileClip(video_path) as video_clip:
            audio_clip = video_clip.audio
            if audio_clip:
                audio_clip.write_audiofile(audio_path, codec='pcm_s16le', logger=None) # logger=None pour moins de verbosité
                audio_clip.close()
                print(f"Audio extrait et sauvegardé dans : {audio_path}")
                return audio_path
            else:
                print("Aucune piste audio trouvée dans la vidéo.")
                return None
    except Exception as e:
        print(f"Erreur lors de l'extraction de l'audio : {e}")
        if os.path.exists(audio_path): os.remove(audio_path)
        return None

def transcribe_audio(audio_path):
    """Transcrire le fichier audio."""
    if not transcription_pipeline:
        print("Modèle de transcription non chargé. Transcription ignorée.")
        return ""
    if not audio_path or not os.path.exists(audio_path):
        print("Chemin audio invalide ou fichier manquant. Transcription ignorée.")
        return ""
    print(f"Transcription de l'audio : {audio_path}")
    try:
        # Forcer la transcription sur le device choisi (CPU ou GPU)
        result = transcription_pipeline(audio_path)
        transcription = result["text"]
        print("Transcription terminée.")
        return transcription
    except Exception as e:
        print(f"Erreur lors de la transcription audio : {e}")
        return ""

In [ ]:
# --- Fonctions Utilitaires Analyze Content ---
temp_image_path = None
def analyze_content(middle_image_obj, transcribed_text, prompt, max_new_tokens=2048):
    """Analyse l'image (PIL) et le texte avec Qwen-VL."""
    if middle_image_obj is None:
        print("Analyse annulée : image manquante.")
        return json.dumps({"error": "Image non fournie ou invalide."})

    print("Début de l'analyse multimodale...")

    if qwen_vl_model and qwen_vl_processor:
        # --- Exécution ---
        try:
            print("Génération de la réponse par Qwen-VL...")
            # Construire la query pour Qwen2.5-VL 3B
            temp_image_path = generate_unique_filename(TEMP_FOLDER, ".jpg")
            middle_image_obj.save(temp_image_path)


            messages = [
                 {"role": "user", "content": [
                         {"type": "image","image": temp_image_path},
                         {"type": "text", "text": f"{prompt}\n\nTranscription Audio: \"{transcribed_text}\""},
                     ],
                 }
            ]

            # Preparation for inference
            text = qwen_vl_processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

            image_inputs, video_inputs = process_vision_info(messages)

            inputs = qwen_vl_processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            )

            inputs = inputs.to(qwen_vl_model.device)

            # Inference: Generation of the output
            output_ids = qwen_vl_model.generate(**inputs, max_new_tokens=max_new_tokens)
            generated_ids = [out_ids[len(input_ids) :] for input_ids, out_ids in zip(inputs.input_ids, output_ids)]
            output_text_list= qwen_vl_processor.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
            print("Réponse Qwen-VL reçue.")

            if output_text_list:
               analysis_result = output_text_list[0] # Utiliser directement la première chaîne
            else:
                analysis_result = json.dumps({"error": "Réponse du modèle vide."})

        except Exception as e:
            print(f"Erreur lors de l'inférence avec Qwen-VL local : {e}")
            analysis_result = json.dumps({"error": f"Erreur d'inférence Qwen-VL: {e}"})
        finally:
            if os.path.exists(temp_image_path): os.remove(temp_image_path)
            return analysis_result
    else:
        # --- SIMULATION ---
        print("Simulation de l'analyse Qwen-VL.")
        simulated_output = {
            "motivation": "personal", "occasion": "birthday", "recording_location": "living room",
            "number_of_people": "1",
            "people": [{"gender": "male", "age_category": "young_adult", "attire": "casual"}],
            "viral_mood": "happy", "relationship": "friend"
        }
        return json.dumps(simulated_output, indent=2)

def cleanup_files(*args):
    """Supprime les fichiers temporaires spécifiés."""
    for file_path in args:
        if file_path and os.path.exists(file_path):
            try:
                os.remove(file_path)
                print(f"Fichier temporaire supprimé : {file_path}")
            except Exception as e:
                print(f"Erreur lors de la suppression du fichier {file_path}: {e}")

print("Fonctions utilitaires définies.")

Fonctions utilitaires définies.


Upload test video

In [ ]:
DRIVE_VIDEOS_PATH = "/content/drive/MyDrive/Personas_project/Data/videos/Vidday_videos"

def choisir_video_drive():
    """Affiche les vidéos disponibles sur Google Drive et permet à l'utilisateur d'en choisir une."""
    try:
        video_files = [f for f in os.listdir(DRIVE_VIDEOS_PATH) if f.endswith(('.mp4', '.mov', '.avi', '.mkv'))]
        if not video_files:
            print(f"Aucun fichier vidéo trouvé dans le dossier : {DRIVE_VIDEOS_PATH}")
            return None

        print("Vidéos disponibles :")
        for i, filename in enumerate(video_files):
            print(f"{i + 1}. {filename}")

        while True:
            try:
                choix = input("Veuillez entrer le numéro de la vidéo à utiliser : ")
                index = int(choix) - 1
                if 0 <= index < len(video_files):
                    video_filename = video_files[index]
                    video_path = os.path.join(DRIVE_VIDEOS_PATH, video_filename)
                    print(f"Vidéo sélectionnée : {video_path}")
                    return video_path
                else:
                    print("Choix invalide. Veuillez entrer un numéro de la liste.")
            except ValueError:
                print("Entrée invalide. Veuillez entrer un numéro.")
    except FileNotFoundError:
        print(f"Le dossier spécifié est introuvable : {DRIVE_VIDEOS_PATH}")
        return None
    except Exception as e:
        print(f"Une erreur est survenue lors de la lecture du dossier : {e}")
        return None

In [ ]:
video_path = choisir_video_drive()

Vidéos disponibles :
1. Birthday_twin.mp4
2. Fathersday_boy.mp4
3. Graduation_1.mp4
4. Funny_Birthday_Memories.mp4
5. Graduation_family.mp4
6. Teacher_appreciation.mp4
7. Wedding.mp4
8. Wedding_wishes.mp4
9. Birthday_friends.mp4
Veuillez entrer le numéro de la vidéo à utiliser : 1
Vidéo sélectionnée : /content/drive/MyDrive/Personas_project/Data/videos/Vidday_videos/Birthday_twin.mp4


App.py

In [ ]:
# --- Exécuter le processus complet ---
analysis_result_json = None
middle_image_obj = None
audio_file_path = None
transcribed_text = ""

if video_path and os.path.exists(video_path):
    try:
        # 1. Extraire l'image du milieu
        middle_image_obj = extract_middle_frame(video_path)

        # 2. Extraire l'audio
        audio_file_path = extract_audio(video_path)

        # 3. Transcrire l'audio
        if audio_file_path:
            transcribed_text = transcribe_audio(audio_file_path)
            print("-" * 20)
            print(f"Transcription: {transcribed_text}")
            print("-" * 20)
        else:
            print("Pas d'audio à transcrire.")

        # 4. Analyser image et texte
        if middle_image_obj:
            analysis_result_json = analyze_content(middle_image_obj, transcribed_text, prompt)
        else:
             analysis_result_json = json.dumps({"error": "Impossible de procéder à l'analyse sans image."}, indent=2)

    except Exception as e:
        print(f"ERREUR MAJEURE pendant le pipeline d'analyse : {e}")
        analysis_result_json = json.dumps({"error": f"Erreur pipeline: {e}"}, indent=2)
    finally:
        # 5. Nettoyer les fichiers temporaires (vidéo originale uploadée et audio extrait)
        print("Nettoyage des fichiers temporaires...")
        cleanup_files(audio_file_path, temp_image_path)
        print("Nettoyage terminé.")

else:
    print("Chemin vidéo invalide. Impossible de lancer l'analyse.")
    analysis_result_json = json.dumps({"error": "Fichier vidéo non trouvé ou non uploadé."}, indent=2)


# --- Afficher le Résultat Final ---
print("\n" + "="*30 + " RÉSULTAT DE L'ANALYSE " + "="*30)
if analysis_result_json:
    # Imprimer le JSON formaté
    print(analysis_result_json)
    # Vous pouvez aussi le charger en dictionnaire Python si besoin
    # try:
    #    result_dict = json.loads(analysis_result_json)
    #    print("\n(Résultat en tant que dictionnaire Python):")
    #    print(result_dict)
    # except json.JSONDecodeError:
    #    print("\nImpossible de parser le résultat en dictionnaire Python.")

else:
    print("Aucun résultat d'analyse généré.")

print("="*80)

Extraction de l'image du milieu de : /content/drive/MyDrive/Personas_project/Data/videos/Vidday_videos/Birthday_twin.mp4
Image du milieu extraite avec succès.
Extraction de l'audio de : /content/drive/MyDrive/Personas_project/Data/videos/Vidday_videos/Birthday_twin.mp4
Audio extrait et sauvegardé dans : /content/drive/MyDrive/Personas_project/Data/temp_files/71c63069-4918-4720-a1fe-c0a25847ad9f.wav
Transcription de l'audio : /content/drive/MyDrive/Personas_project/Data/temp_files/71c63069-4918-4720-a1fe-c0a25847ad9f.wav


  warnings.warn(

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Transcription terminée.
--------------------
Transcription:  From your very old personal birthday to in, thank you for being an amazing leader and a wonderful human being. We're so happy to have you on this planet, happy birthday and enjoy your special day.
--------------------
Début de l'analyse multimodale...
Génération de la réponse par Qwen-VL...
Réponse Qwen-VL reçue.
Nettoyage des fichiers temporaires...
Fichier temporaire supprimé : /content/drive/MyDrive/Personas_project/Data/temp_files/71c63069-4918-4720-a1fe-c0a25847ad9f.wav
Nettoyage terminé.

============================== RÉSULTAT DE L'ANALYSE ==============================
```json
{
  "number_of_people": "1",
  "people": [
    {
      "gender": "female",
      "age": "25–34 years",
      "attire": "casual"
    }
  ],
  "recording_location": "outdoor",
  "motivation": "personal",
  "occasion": "birthday",
  "viral_mood": "happy",
  "relationship": "friend"
}
```
